In [1]:
from collections import Counter
import math
import numpy as np
import tensorflow as tf

In [2]:
class Tokenizer():
    """
        分词器
    """
    def __init__(self,token_dict):
        # 词 -> ID 
        self.token_dict = token_dict
        # ID -> 词
        self.token_dict_rev = {value:key for key,value in self.token_dict.items()}
        # 词汇表大小
        self.vocab_size = len(self.token_dict)
        
        
    def id_to_token(self,token_id):
        """
        给定一个编号，查找词汇表中对应的词
        :param token_id: 带查找词的编号
        :return: 编号对应的词
        """
        return self.token_dict_rev[token_id]
    
    def token_to_id(self,token):
        """
        给定一个词，查找它在词汇表中的编号
        未找到则返回低频词[UNK]的编号
        :param token: 带查找编号的词
        :return: 词的编号
        """
        return self.token_dict.get(token,self.token_dict['[UNK]'])
        
    def encode(self,tokens):
        """
        给定一个字符串s，在头尾分别加上标记开始和结束的特殊字符，并将它转成对应的编号序列
        :param tokens: 待编码字符串
        :return: 编号序列
        """
        # 加上开始标记
        token_ids = [self.token_to_id('[CLS]'), ]
        # 加入字符串编号序列
        for token in tokens:
            token_ids.append(self.token_to_id(token))
            
        # 加上结束标记
        token_ids.append(self.token_to_id('[SEP]'))
        return token_ids
    
    def decode(self,token_ids):
        """
        给定一个编号序列，将它解码成字符串
        :param token_ids: 待解码的编号序列
        :return: 解码出的字符串
        """
        # 起止标记字符特殊处理
        spec_tokens = ['[CLS]','[SEP]']
        
        # 保存解码出的字符list
        tokens = []
        for token_id in token_ids:
            token = self.id_to_token(token_id)
            if token in spec_tokens:
                continue
                
            tokens.append(token)
            
        #拼接字符串
        return ''.join(tokens)    

In [3]:
# 禁用词
DISALLOWED_WORDS= ['（', '）', '(', ')', '__', '《', '》', '【', '】', '[', ']']

In [4]:
# 数据集路径
DATASET_PATH = './data/poetry.txt'

In [5]:
# 每个epoch训练完成后，随机生成SHOW_NUM首古诗作为展示
SHOW_NUM = 5
# 最佳模型保存路径
BEST_MODEL_PATH = './out/best_model.h5'

In [6]:
# 句子最大长度
MAX_LEN = 64

In [7]:
# 最小词频
MIN_WORD_FREQUENCY= 8

In [8]:
# 训练的batch_size
BATCH_SIZE= 16

In [9]:
# 共训练多少个epoch
TRAIN_EPOCHS = 2

In [10]:
disallowed_words = DISALLOWED_WORDS
max_len = MAX_LEN
min_word_frequency = MIN_WORD_FREQUENCY
batch_size = BATCH_SIZE

In [11]:
# 加载数据集
with open(DATASET_PATH,'r',encoding='utf-8') as f:
    lines = f.readlines()
    lines = [line.replace("：",":") for line in lines]

In [12]:
lines[0]

'首春:寒随穷律变，春逐鸟声开。初风飘带柳，晚雪间花梅。碧林青旧竹，绿沼翠新苔。芝田初雁去，绮树巧莺来。\n'

In [13]:
# 数据集列表
poetry= []
# 逐行处理读取到的数据
for line in lines:
    if line.count(":")!=1:
        continue
    # 分割后半部分
    _,last_part = line.split(":")
    ignore_flag = False
    for dis_word in disallowed_words:
        if dis_word in last_part:
            ignore_flag =True
            break
    if ignore_flag:
        continue
    # 长度不能超过最大长度
    if len(last_part) > max_len -2:
        continue
    poetry.append(last_part.replace("\n",""))

In [14]:
# 统计词频
counter = Counter()
for line in poetry:
    counter.update(line)

In [15]:
# 过滤掉低频的词
_tokens = [(token,count) for token,count in counter.items() if count >= min_word_frequency]

In [16]:
# 按词频排序，只保留词列表
_tokens = sorted(_tokens, key= lambda x :-x[1])

In [17]:
# 去掉词频 只保留词列表
_tokens = [token for token,count in _tokens]

In [18]:
# 将特殊词和数据集拼接起来
_tokens = ['[PAD]','[UNK]','[CLS]','[SEP]']+_tokens

In [19]:
range(len(_tokens))

range(0, 3434)

In [20]:
zip(_tokens,range(len(_tokens)))

In [21]:
# 创建字典 token -> id 的关系
token_id_dict = dict(zip(_tokens,range(len(_tokens))))

In [22]:
# 使用词典重建分词器
tokenizer = Tokenizer(token_id_dict)

In [23]:
tokenizer

In [24]:
# 混洗数据
np.random.shuffle(poetry)

In [25]:
class PoetryDataGenerator():
    """
    训练数据集生成
    """
    def __init__(self,data,random=False):
        #数据集
        self.data = data
        # batch_size
        self.batch_size= batch_size
        # 每个epoch迭代的步数
        self.steps = int(math.floor(len(self.data) / self.batch_size))
        # 每个epoch开始的时候是否随机混洗
        self.random = random
        
    def sequence_padding(self,data,length=None,padding=None):
        """
        将给定数据填充到相同长度
        :param data: 待填充数据
        :param length: 填充后的长度，不传递此参数则使用data中的最大长度
        :param padding: 用于填充的数据，不传递此参数则使用[PAD]的对应编号
        :return: 填充后的数据
        """
        
        # 计算填充长度
        if length is None:
            length = max(map(len,data))
            
        # 计算填充数据
        if padding is None:
            padding = tokenizer.token_to_id('[PAD]')
            
        # 开始填充
        outputs = []
        
        for line in data:
            padding_length = length -len(line)
            # 不足就填充
            if padding_length >0:
                outputs.append(np.concatenate([line,[padding] *padding_length]))
            # 超过就截断
            else:
                outputs.append(line[:length])
                
        return np.array(outputs)
    
    def __len__(self):
        return self.steps
    
    
    def __iter__(self):
        total = len(self.data)
        
        # 是否孙吉混洗
        if self.random:
            np.random.shuffle(self.data)
            
        # 迭代一个epoch，每次yield 一个batch
        for start in range(0,total,self.batch_size):
            end = min(start+self.batch_size,total)
            batch_data = []
            
            #逐一对古诗进行编码
            for single_data in self.data[start:end]:
                batch_data.append(tokenizer.encode(single_data))
                
            # 填充为相同长度
            batch_data = self.sequence_padding(batch_data)
            
            # yield x,y
            yield batch_data[:,:-1],tf.one_hot(batch_data[:,1:], tokenizer.vocab_size)
            
            del batch_data
            
    def for_fit(self):
        """
        创建一个生成器，用于训练
        """
        while True:
            yield from self.__iter__()

In [26]:
a =['1','2',3,4,5,6]

In [27]:
a[-1]

6

In [28]:
a[:-1]

['1', '2', 3, 4, 5]

In [37]:
def generate_poetry(tokenizer,model,head):
    """
    随机生成一首藏头诗
    :param tokenizer: 分词器
    :param model: 用于生成古诗的模型
    :param head: 藏头诗的头
    :return: 一个字符串，表示一首古诗
    """
    # 使用空串初始化token_ids ,加入[CLS]
    token_ids = tokenizer.encode('')
    token_ids = token_ids[:-1]
    
    # 标点符号
    punctuations = ['，','。']
    punctuations_ids = [tokenizer.token_to_id(token) for token in punctuations]
    
    # 村换生成诗的list
    poetry =[]
    
    # 对于藏头诗的每一个字。都生成一个短句
    for ch in head:
        #先记录这个字
        poetry.append(ch)
        # 将藏头字转为id 
        token_id = tokenizer.token_to_id(ch)
        # 加入进列表
        token_ids.append(token_id)
        
        # 生成短句
        while True:
            #进行预测,只保留第一个样例（我们输入的样例数只有1）的、最后一个token的预测的、不包含[PAD][UNK][CLS]的概率分布
            output = model(np.array([token_ids,], dtype=np.int32))
            _probas = output.numpy()[0,-1,3:]
            del output
            
            #按照出现概率，对所有的token倒序排列
            p_args = _probas.argsort()[::-1][:100]
            #排列后的概率顺序
            p = _probas[p_args]
            
            #对概率归一
            p = p /sum(p)
            
            #再按照预测出的概率，随机选择一个词作为预测结果
            target_index = np.random.choice(len(p), p=p)
            target = p_args[target_index] +3
            
            #保存
            token_ids.append(target)
            # 只有不是特殊字符时，才保存到poetry中
            if target >3:
                poetry.append(tokenizer.id_to_token(target))
                
            if target in punctuations_ids:
                break
    return ''.join(poetry)

In [38]:
help(tf.keras.layers.Input)

Help on function Input in module keras.engine.input_layer:

Input(shape=None, batch_size=None, name=None, dtype=None, sparse=None, tensor=None, ragged=None, type_spec=None, **kwargs)
    `Input()` is used to instantiate a Keras tensor.
    
    A Keras tensor is a symbolic tensor-like object,
    which we augment with certain attributes that allow us to build a Keras model
    just by knowing the inputs and outputs of the model.
    
    For instance, if `a`, `b` and `c` are Keras tensors,
    it becomes possible to do:
    `model = Model(input=[a, b], output=c)`
    
    Args:
        shape: A shape tuple (integers), not including the batch size.
            For instance, `shape=(32,)` indicates that the expected input
            will be batches of 32-dimensional vectors. Elements of this tuple
            can be None; 'None' elements represent dimensions where the shape is
            not known.
        batch_size: optional static batch size (integer).
        name: An optional name

In [39]:
"""
构建lstm模型
"""
# model = tf.keras.Sequential()
# #不定长度的输入
# model.add(tf.keras.layers.Input(None,))
# # 词嵌入层
# model.add(tf.keras.layers.Embedding(input_dim=tokenizer.vocab_size, output_dim=128))
# # 第一个lstm层
# model.add(tf.keras.layers.LSTM(128,dropout=0.5,return_sequence=True))
# model.add(tf.keras.layers.LSTM(128,dropout=0.5,return_sequence=True))

# # 对每一个时间点输出都做softmax, 预测下一个词的概率
# model.add(tf.keras.layers.TimeDistributed(
#     tf.keras.layers.Dense(tokenizer.vocab_size,activation='softmax')
# ))
model = tf.keras.Sequential([
    # 不定长度的输入
    tf.keras.layers.Input((None,)),
    # 词嵌入层
    tf.keras.layers.Embedding(input_dim=tokenizer.vocab_size, output_dim=128),
    # 第一个LSTM层，返回序列作为下一层的输入
    tf.keras.layers.LSTM(128, dropout=0.5, return_sequences=True),
    # 第二个LSTM层，返回序列作为下一层的输入
    tf.keras.layers.LSTM(128, dropout=0.5, return_sequences=True),
    # 对每一个时间点的输出都做softmax，预测下一个词的概率
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(tokenizer.vocab_size, activation='softmax')),
])

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         439552    
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 3434)        442986    
Total params: 1,145,706
Trainable params: 1,145,706
Non-trainable params: 0
_________________________________________________________________


In [41]:
# 配置优化器和损失函数
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.categorical_crossentropy)

In [42]:
"""
模型训练
"""
class Evaluate(tf.keras.callbacks.Callback):
    """
    训练过程评估，在每个epoch训练完成后，保留最优权重，并随机生成SHOW_NUM首古诗展示
    """

    def __init__(self):
        super().__init__()
        # 给loss赋一个较大的初始值
        self.lowest = 1e10
        
    def on_epoch_end(self,epoch,logs=None):
        #在每个epochi训练完成后调用
        #如果当前loss更低，则保存当前模型参数
        if logs['loss'] <= self.lowest:
            self.lowest = logs['loss']
            model.save(BEST_MODEL_PATH)
            
        #随机生成古诗测试，查看效果
        print("生成测试")
        
        for i in range(SHOW_NUM):
            print(generate_poetry(tokenizer,model,head="春花秋月"))

In [43]:
# 创建数据集
data_generator = PoetryDataGenerator(poetry,random=True)

In [45]:
model.fit(
    data_generator.for_fit(),
    steps_per_epoch=data_generator.steps,
    workers=-1,
    epochs=10,
    callbacks=[Evaluate()]
)

Epoch 1/10
1534/1534 [==============================] - 30s 20ms/step - loss: 4.1433
生成测试
春柳无何到，花年万里流。秋南山阳在，月日柳峰间。
春阳楚天去，花里暮山边。秋水夜窗鸟，月天拂野连。
春峰春上月来，花柳几日半花来。秋流天夜吹条树，月雨新山似处头。
春夜随云远一秦，花城吹处问人看。秋朝唯可见头者，月梦花僧五首朝。
春山春风，花前长故尘。秋中未相酒，月夜两湘云。
Epoch 2/10
1534/1534 [==============================] - 30s 20ms/step - loss: 3.9983
生成测试
春色三城树，花秋望暮村。秋潮无人事，月叶得何心。
春风万里过，花鸟上时惊。秋水深千里，月啼白岸声。
春月花亭好，花州夕岁心。秋风不思处，月向梦中时。
春草有时路，花江似远乡。秋边闻鸟夕，月水水流头。
春上多春野夜人，花西犹到客家知。秋风不见高桥去，月日清天吹更来。
Epoch 3/10
1534/1534 [==============================] - 30s 20ms/step - loss: 3.9176
生成测试
春下云沙一客红，花生不到月西烟。秋宵吹曲人寻语，月日人逢落下回。
春风出酒远，花发暮飞明。秋影归初晓，月空见汉人。
春山临水寺，花落落阳舟。秋霁入秋望，月无空满空。
春暖暮梅花，花山绕井连。秋山长雁远，月树隔秋云。
春边尽别醉，花岸照中春。秋水秋风水，月光花绿微。
Epoch 4/10
1534/1534 [==============================] - 30s 20ms/step - loss: 3.8707
生成测试
春日萧楼天，花晴几渐赊。秋风入竹叶，月静逐年台。
春流有竹树，花中出客津。秋风何故景，月有见孤关。
春光千阙水，花雨树天关。秋郭山河水，月深照寺深。
春光不知时，花上雪溪边。秋吹九云水，月花深夜城。
春风正有路，花处复经年。秋气临风树，月开窗早鸣。
Epoch 5/10
1534/1534 [==============================] - 31s 20ms/step - loss: 3.8251
生成测试
春欲相无事，花前复不游。秋风起不似，月雨

In [47]:
"""
输入关键字，生成藏头诗
"""



# 加载训练好的模型
model = tf.keras.models.load_model(BEST_MODEL_PATH)

keywords = input('输入关键字:\n')


# 生成藏头诗
for i in range(SHOW_NUM):
    print(generate_poetry(tokenizer, model, head=keywords),'\n')

输入关键字:
 高智勇


高斋有春物，智气喜无情。勇在风前水， 

高僧何日无人绪，智土从君合了存。勇得还同能有苦， 

高阶独不极，智树不成开。勇得新兵薄， 

高步飞晴见雪声，智来不见又空悲。勇书自说堪闲泪， 

高路江林远，智公已故川。勇章怜旧事， 

